In [1]:
# Implementation of HRED model in PyTorch
# Paper : https://arxiv.org/abs/1507.04808
# python hred_pytorch.py <training_data> <dictionary>
# https://github.com/koustuvsinha/hred-py/blob/master/hred_pytorch.py

import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
#from torch.optim.lr_scheduler import StepLR
import torch.nn.functional as F
import json
import pickle as pkl
import random
import sys
import time
import math
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import numpy as np
import re
import os.path
from tqdm import tqdm
import nltk
from nltk import word_tokenize
import string

use_cuda = torch.cuda.is_available()
print(use_cuda)

import seaborn as sns
import pandas as pd
% matplotlib inline
sns.set(style='whitegrid', palette='muted', font_scale=1.5)
pd.set_option('display.max_colwidth', 300) #widen pandas rows display

False


In [5]:
q_a_df = pd.read_csv('data/q_a_all.csv')
q_a_df.reset_index(inplace=True)
q_a_df.tail()

,index,Unnamed: 0,Unnamed: 0.1,Q,A,SubjectOfBusinessTitle,FloorLanguage,date,personSpeaking
27727,27727,27727,27727,"Mr. Speaker, the Prime Minister allowed a convicted terrorist to attend an official Government of Canada event in India. Someone in the Prime Minister's Office then ordered the national security adviser to tell the media that the Indian government was responsible for this embarrassment. Yesterda...","Mr. Speaker, of course, at no point did members of the public service ever reveal classified information to the media, nor would they. The issue is that the Leader of the Opposition so wants to be able to play political games with this issue that he plugs his ears, refuses to know the truth, and...",Public Safety,EN,2018-03-27,"Right Hon. Justin Trudeau (Prime Minister, Lib.)"
27728,27728,27728,27728,"Mr. Speaker, the only reason why the Prime Minister is offering me a briefing of classified information in my capacity as a privy councillor is to prevent me from asking questions about his disastrous trip to India and the cover-up that he helped orchestrate. All we are asking for is that the sa...","Mr. Speaker, only Stephen Harper's Conservative Party would think that giving information to the media is somehow hiding information from Canadians. The question is: why does the Leader of the Opposition not want to know the truth? Canadians understand that when it comes to intelligence issues a...",Public Safety,EN,2018-03-27,"Hon. Andrew Scheer (Leader of the Opposition, CPC)"
27729,27729,27729,27729,"Mr. Speaker, in a healthy and prosperous society, it is essential to ensure that everyone can participate in the economy on a level playing field. Can the Minister of Status of Women tell us what we are doing in budget 2018 to further empower women so that they have equal opportunities to work i...","Mr. Speaker, I thank my colleague from Vimy for her leadership in advancing gender equality. By investing in women, we will improve the economy for everyone. [English]\n Budget 2018 includes several measures to close the gender wage gap, including an investment of $1.65 billion for ...",Status of Women,EN,2018-03-27,"Mrs. Eva Nassif (Vimy, Lib.)"
27730,27730,27730,27730,"Mr. Speaker, the government has a troubling history that every time its members get pushed on questions that show the fact that they do not have a good answer, they resort to name calling. Yesterday at the finance committee I was questioning the minister with respect to his own record on promoti...","Mr. Speaker, I was at the committee meeting yesterday. It is unfortunate that the opposition members make such personal attacks, when they have an opportunity to discuss budget 2018 and provide constructive criticism. They do everything but talk about the budget while we are tackling the followi...",Status of Women,EN,2018-03-27,"Hon. Lisa Raitt (Milton, CPC)"
27731,27731,27731,27731,"Mr. Speaker, there is no attack in saying the facts. One, the minister never once sought to ensure he had proper representation on his private sector board. Two, the finance department is woefully lacking women in senior positions by 4:11. Three, in his own office there is only one woman who is ...","Mr. Speaker, we often see in the House a type of selective amnesia. I would like to remind the House that the cabinet of the previous prime minister, Stephen Harper, was far from gender-balanced. That is the example set by the Conservatives for Canadians. Budget 2018 provides for shared parental...",Status of Women,EN,2018-03-27,"Mr. Joël Lightbound (Parliamentary Secretary to the Minister of Finance, Lib.)"


In [12]:
'''Load the training dataset'''

EOS_token = 'eos'
SOS_token = 'sos'
UNK_token = '<unk>'
NUM_token = '<number>'

q_a_df = pd.read_csv('data/q_a_all.csv')


def text_to_tokens(text_seqs):
    tokens = re.sub(r'[^\x00-\x7f]', u'', text_seqs)
    return nltk.word_tokenize(tokens)
                              

def clean(x):
    x = x.replace('BOS', '')
    x = x.replace('EOS', '')
    return x.lower()

def text_to_sents(text_seqs):
    tokens = re.sub(r'[^\x00-\x7f]', u'', text_seqs)
    return nltk.sent_tokenize(tokens) # this gives us a list of sentences



q_a_df['Q'] = q_a_df['Q'].apply(clean)
q_a_df['A'] = q_a_df['A'].apply(clean)


q_a_df['Q'] = q_a_df['Q'].apply(lambda x: u' '.join(['sos '+p+' eos' for p in text_to_sents(x)[-2:]]))
q_a_df['A'] = q_a_df['A'].apply(lambda x: u' '.join(['sos '+p+' eos' for p in text_to_sents(x)[:2]]))

# q_a_df['Q'] = q_a_df['Q'].apply(lambda x: u' '.join(['sos '+p+' eos' for p in text_to_sents(x)]))
# q_a_df['A'] = q_a_df['A'].apply(lambda x: u' '.join(['sos '+p+' eos' for p in text_to_sents(x)]))

# q_a_df = q_a_df[:20000]

q_count = 0
a_count = 0
for i, row in q_a_df.iterrows():
    q_count += len(row['Q'].split())
    a_count += len(row['A'].split())


print('Q average length', q_count / len(q_a_df))
print('A average length', a_count / len(q_a_df))

q_a_df[['Q','A']].to_csv('Q_A.tsv', sep='\t', encoding='utf-8', index=False)
q_a_df[['Q','A']].tail()


Q average length 41.07615750757248
A average length 41.27037357565268


,Q,A
27727,"sos yesterday, the prime minister and the public safety minister suggested that the content of that media briefing was classified. eos sos can the prime minister inform us who in his office helped orchestrate the release of classified information to the media? eos","sos mr. speaker, of course, at no point did members of the public service ever reveal classified information to the media, nor would they. eos sos the issue is that the leader of the opposition so wants to be able to play political games with this issue that he plugs his ears, refuses to know th..."
27728,"sos the prime minister has just affirmed that classified information was not provided to the media. eos sos why, then, can that same information not be provided to the house? eos","sos mr. speaker, only stephen harper's conservative party would think that giving information to the media is somehow hiding information from canadians. eos sos the question is: why does the leader of the opposition not want to know the truth? eos"
27729,"sos mr. speaker, in a healthy and prosperous society, it is essential to ensure that everyone can participate in the economy on a level playing field. eos sos can the minister of status of women tell us what we are doing in budget2018 to further empower women so that they have equal opportunitie...","sos mr. speaker, i thank my colleague from vimy for her leadership in advancing gender equality. eos sos by investing in women, we will improve the economy for everyone. eos"
27730,sos it is unacceptable language. eos sos could the minister please clarify what he was trying to say? eos,"sos mr. speaker, i was at the committee meeting yesterday. eos sos it is unfortunate that the opposition members make such personal attacks, when they have an opportunity to discuss budget 2018 and provide constructive criticism. eos"
27731,sos that is his choice. eos sos i have every right to ask questions that make the government uncomfortable and i am going to continue to do it whether those members like it or not. eos,"sos mr. speaker, we often see in the house a type of selective amnesia. eos sos i would like to remind the house that the cabinet of the previous prime minister, stephen harper, was far from gender-balanced. eos"


In [13]:
corpus = []
for i, row in q_a_df.iterrows():
    corpus.append(text_to_tokens(row['Q']))
    corpus.append(text_to_tokens(row['A']))
    
corpus = [item for sublist in corpus for item in sublist]
# print(corpus[:100])
print('number of words in Q and A:', len(corpus))

from collections import Counter
counts = Counter(corpus)
# print(counts.most_common(10000))

    
freq_tokens = [SOS_token, EOS_token, UNK_token, NUM_token] + list(string.punctuation)
for token in counts.most_common(10000):
    freq_tokens.append(token[0])

freq_tokens = list(set(freq_tokens))
# freq_tokens[:1000]

number of words in Q and A: 2521348


In [14]:
# helper functions to read the corpuse, build the pairs, and dictionary.
SOS_token_id = 0
EOS_token_id = 1
MAX_LENGTH = 50 # length of Q and A 

def is_number_repl_isdigit(s):
    """ Returns True is string is a number. """
    return s.replace('.','',1).isdigit()


from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: SOS_token, 1: EOS_token, 2: UNK_token}
        self.n_words = 3  # Count SOS and EOS

    def addSentence(self, sentence):
#         for word in sentence.split(' '):
        for word in nltk.word_tokenize(sentence):
            if word not in freq_tokens:
                word = UNK_token
            if is_number_repl_isdigit(word):
                word = NUM_token
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('Q_A.tsv', encoding='utf-8').read().strip().split('\n')
#     print(lines[1])

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines[1:]]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

# input_lang, output_lang, pairs = readLangs('Q', 'A')
# print(pairs[0])

def filterPair(p):
    if len(p) != 2:
        return False
    return len(p[0].split(' ')) < MAX_LENGTH and len(p[1].split(' ')) < MAX_LENGTH

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('Q', 'A', False)
# print(pairs)
print(random.choice(pairs))

Reading lines...
Read 28039 sentence pairs
Trimmed to 12861 sentence pairs
Counting words...
Counted words:
Q 8087
A 7374
['sos is she going to use the same sleeping bag defence as the minister responsible for conflicts of interest ? eos sos as long as she is wasting public funds did the minister at least take the shampoo samples from the room ? eos', 'sos mr . speaker as i have already said our government expects our ministers to use taxpayers money properly . eos sos the minister reimbursed the cost of changing hotels . eos']


In [15]:

# groups = []
# word2id = {}
# id2word = {}
print('input_lang.index2word',len(input_lang.index2word))
print('output_lang.index2word', len(output_lang.index2word))
groups = pairs
word2id = {**input_lang.word2index, **output_lang.word2index}
id2word = {**input_lang.index2word, **output_lang.index2word}


print('pairs', len(pairs))
print('word2id len', len(word2id))


input_lang.index2word 8087
output_lang.index2word 7374
pairs 12861
word2id len 8905


In [22]:
%%time


from sklearn import preprocessing
KERAS_DATASETS_DIR = '/home/paperspace/.keras/datasets/'
GLOVE_FILE = 'glove.840B.300d.txt'

EMBEDDING_DIM = 300

print("Processing", GLOVE_FILE)

embeddings_index = {}
with open(KERAS_DATASETS_DIR + GLOVE_FILE, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        embedding = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = embedding

print('Word embeddings: %d' % len(embeddings_index))



print('Using input words to initialize the embedding:', input_lang.n_words)
embedding_matrix = np.random.uniform(-1, 1, size=(len(word2id) , EMBEDDING_DIM))
count = 0
for word, i in word2id.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector[:EMBEDDING_DIM]
        count += 1

print('L2 Normalization')

# l2-normalize the samples (rows). 
embedding_matrix = preprocessing.normalize(embedding_matrix, norm='l2')

print('Null word embeddings: %d' % (len(word2id) - count))

Processing glove.840B.300d.txt
Word embeddings: 2196016
Using input words to initialize the embedding: 8087
L2 Normalization
Null word embeddings: 213
CPU times: user 2min 27s, sys: 1.97 s, total: 2min 29s
Wall time: 2min 29s


In [16]:



class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        # Pretrained embedding
#         self.embedding.weight = nn.Parameter(torch.from_numpy(embedding_matrix))
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
#         self.embedding.weight.requires_grad = False

        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        #h0,c0 = hidden
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        #hidden = (h1,c1)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        #cell = Variable(torch.zeros(1,1,self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

class ContextRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1):
        super(ContextRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        #print input
        #output = self.embedding(input).view(1, 1, -1)
        output = input.view(1,1,-1)
        for i in range(self.n_layers):
            output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result


# for hred, decoder should also take the context vector and multiply that with the hidden state to form the new hidden
# state

class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_output, encoder_outputs,context):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)))
        attn_applied = torch.bmm(attn_weights.unsqueeze(0), # bmm - matmul
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        # inputs are concatenation of previous output and context
        for i in range(self.n_layers):
            output = F.relu(output)
            output = torch.cat((output,context),0)
            output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0])) # log softmax is done for NLL Criterion. We could use CrossEntropyLoss to avoid calculating this
        return output, hidden, attn_weights

    def initHidden(self):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_cuda:
            return result.cuda()
        else:
            return result

In [17]:
word2id[EOS_token]

17

In [18]:


teacher_forcing_ratio = 0.5

# larger beam widths result in better performance of a model as the multiple candidate sequences 
# increase the likelihood of better matching a target sequence.
# This increased performance results in a decrease in decoding speed.

class HRED_QA(object):
    def __init__(self,
            groups=None,      # qa data in groups
            dictionary=None,  # should be a HRED model preprocessed dictionary
            id2word=None,
            word2id=None,
            encoder_file=None,
            decoder_file=None,
            context_file=None,
            teacher_forcing_ratio=0.5,
            hidden_size=512,
            beam=5, 
            max_sentence_length=MAX_LENGTH,
            context_layers = 1,
            attention_layers = 1,
            decoder_layers = 1,
            learning_rate = 0.0001
            ):
        self.groups = groups
        self.dictionary = dictionary
        self.word2id = word2id
        self.id2word = id2word
        self.encoder_file = encoder_file
        self.decoder_file = decoder_file
        self.context_file = context_file
        self.teacher_forcing_ratio = teacher_forcing_ratio
        self.hidden_size = hidden_size
        self.beam = beam
        self.max_sentence_length = max_sentence_length
        self.context_layers = context_layers
        self.attention_layers = attention_layers
        self.decoder_layers = decoder_layers
        self.learning_rate = learning_rate
        self.encoder_model = None
        self.decoder_model = None
        self.context_model = None
        
        self.EOS_token = self.word2id[EOS_token]
        self.SOS_token = self.word2id[SOS_token]

        # load word2id if not present but dictionary is string
#         if self.dictionary and type(self.dictionary) == str and not self.word2id:
#             dt = pkl.load(open(self.dictionary,'r'))
#             self.word2id = {d[0]:d[1] for d in dt}
#             self.id2word = {d[1]:d[0] for d in dt}
#             self.EOS_token = self.word2id['</s>']
#             self.SOS_token = self.word2id['</d>']
#             self.dictionary = dt

        self.create_or_load_models()

    # cerate models if they are none
    # load models if they are string
    def create_or_load_models(self):
        encoder_model = EncoderRNN(len(self.word2id.keys()), self.hidden_size)
        decoder_model = AttnDecoderRNN(self.hidden_size, 
                len(self.word2id.keys()),self.attention_layers, dropout_p=0.1)
        context_model = ContextRNN(self.hidden_size,len(self.word2id.keys()))

        if self.encoder_file and type(self.encoder_file)==str and os.path.exists(self.encoder_file):
            print('loading encorder')
            encoder_model.load_state_dict(torch.load(self.encoder_file))

        if self.decoder_file and type(self.decoder_file)==str and os.path.exists(self.decoder_file):
            decoder_model.load_state_dict(torch.load(self.decoder_file))

        if self.context_file and type(self.context_file)==str and os.path.exists(self.context_file):
            context_model.load_state_dict(torch.load(self.context_file))

        if use_cuda:
            encoder_model = encoder_model.cuda()
            decoder_model = decoder_model.cuda()
            context_model = context_model.cuda()

        self.encoder_model = encoder_model
        self.decoder_model = decoder_model
        self.context_model = context_model
        return self.encoder_model, self.decoder_model, self.context_model


    # for hred, train should take the context of the previous turn
    # should return current loss as well as context representation

    def train(self,input_variable, target_variable,
            encoder, decoder, context, context_hidden,
            encoder_optimizer, decoder_optimizer, criterion,
            last,max_length=None):

        max_length=self.max_sentence_length
        encoder_hidden = encoder.initHidden()

        #encoder_optimizer.zero_grad() # pytorch accumulates gradients, so zero grad clears them up.
        #decoder_optimizer.zero_grad()

        input_length = input_variable.size()[0]
        target_length = target_variable.size()[0]

        encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
        encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(
                input_variable[ei], encoder_hidden)
            encoder_outputs[ei] = encoder_output[0][0]

        decoder_input = Variable(torch.LongTensor([[self.SOS_token]]))
        decoder_input = decoder_input.cuda() if use_cuda else decoder_input

        decoder_hidden = encoder_hidden
        
        # calculate context
        context_output,context_hidden = context(encoder_output,context_hidden)

        use_teacher_forcing = random.random() < self.teacher_forcing_ratio

        if use_teacher_forcing:
            # Teacher forcing: Feed the target as the next input
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_output, encoder_outputs,context_hidden)
#                 loss += criterion(decoder_output[0], target_variable[di])
                loss += criterion(decoder_output, target_variable[di])
                decoder_input = target_variable[di]  # Teacher forcing

        else:

            # Without teacher forcing: use its own predictions as the next input
            for di in range(target_length):
                decoder_output, decoder_hidden, decoder_attention = decoder(
                    decoder_input, decoder_hidden, encoder_output, encoder_outputs,context_hidden)
                topv, topi = decoder_output.data.topk(1)
                ni = topi[0][0]

                decoder_input = Variable(torch.LongTensor([[ni]]))
                decoder_input = decoder_input.cuda() if use_cuda else decoder_input

                # only calculate loss if its the last turn
                if last:
#                     loss += criterion(decoder_output[0], target_variable[di])
                    loss += criterion(decoder_output, target_variable[di])
                if ni == self.EOS_token:
                    break

        if last:
            loss.backward()

        #encoder_optimizer.step()
        #decoder_optimizer.step()

        if last:
            return loss.data[0] / target_length, context_hidden
        else:
            return context_hidden

    def indexesFromSentence(self,word2id,sentence):
        return [word2id.get(word,word2id[UNK_token]) for word in sentence.split(' ') if len(word) > 0]


    def variableFromSentence(self, sentence=None,indexes=None):
        indexes = self.indexesFromSentence(self.word2id, sentence)
        indexes.append(self.EOS_token)
        #print len(indexes)
        result = Variable(torch.LongTensor(indexes).view(-1, 1),requires_grad=False)
        if use_cuda:
            return result.cuda()
        else:
            return result

    def variablesFromPair(self,pair):
        input_variable = self.variableFromSentence(indexes=pair[0])
        target_variable = self.variableFromSentence(indexes=pair[1])
        return (input_variable, target_variable)

    # return variables from group
    def variablesFromGroup(self,group):
        variables = [self.variableFromSentence(sentence=p) for p in group]
        return variables

    # training should proceed over each set of dialogs
    # which should be in variable groups = [u1,u2,u3...un]
    def trainIters(self,encoder=None, decoder=None, n_epochs=1000,
            context=None, print_every=500, plot_every=100, evaluate_every=1000, 
            learning_rate=None):

        encoder = self.encoder_model
        decoder = self.decoder_model
        context = self.context_model
        learning_rate = self.learning_rate
        start = time.time()
        plot_losses = []
        print_loss_total = 0  # Reset every print_every
        plot_loss_total = 0  # Reset every plot_every

        learning_rate = 0.0001 # good for Adam
        encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
        decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate)
        context_optimizer = optim.Adam(context.parameters(), lr=learning_rate)

        # TODO: experiment with decayed learning rate when the api is available
        #enc_scheduler = StepLR(encoder_optimizer,step_size=3000,gamma=0.7)
        #dec_scheduler = StepLR(decoder_optimizer,step_size=3000,gamma=0.7)
        #con_scheduler = StepLR(context_optimizer,step_size=3000,gamma=0.7) 
        criterion = nn.NLLLoss()

        print ("training started")
#         iter = 0
#         while True:
        for iter in tqdm(range(1, n_epochs + 1)):
#             iter +=1
            #training_pair = training_pairs[iter - 1]
            training_group = self.variablesFromGroup(random.choice(self.groups))
            #print len(training_group)
            context_hidden = context.initHidden()
            context_optimizer.zero_grad()
            encoder_optimizer.zero_grad() # pytorch accumulates gradients, so zero grad clears them up.
            decoder_optimizer.zero_grad()
            for i in range(0, len(training_group)-1):
                input_variable = training_group[i]
                target_variable = training_group[i+1]
                last = False
                if i + 1 == len(training_group) - 1:
                    last = True

                if last:
                    loss,context_hidden = self.train(input_variable, target_variable, encoder,
                                                     decoder, context, context_hidden, encoder_optimizer,
                                                     decoder_optimizer, criterion, last)
                    print_loss_total += loss
                    plot_loss_total += loss
                    encoder_optimizer.step()
                    decoder_optimizer.step()
                    context_optimizer.step()
                else:
                    context_hidden = self.train(input_variable, target_variable, encoder, 
                                                decoder, context, context_hidden, encoder_optimizer, 
                                                decoder_optimizer, criterion, last)

            if iter % print_every == 0:
                print_loss_avg = print_loss_total / print_every
                print_loss_total = 0
                print('steps %d loss %.4f' % (iter,print_loss_avg))

            if iter % (print_every * 3) == 0:
                # save models
                print ("saving models")
                torch.save(encoder.state_dict(), self.encoder_file)
                torch.save(decoder.state_dict(), self.decoder_file)
                torch.save(context.state_dict(), self.context_file)

            if iter % plot_every == 0:
                plot_loss_avg = plot_loss_total / plot_every
                plot_losses.append(plot_loss_avg)
                plot_loss_total = 0

            if iter % evaluate_every == 0:
                self.evaluateRandomly(encoder,decoder,context)
                self.generateResponse(text='liberal tax plans')

            #enc_scheduler.step()
            #dec_scheduler.step()
            #con_scheduler.step()
        showPlot(plot_losses)

    def evaluate(self, encoder, decoder, context, sentences, max_length=None, beam=1):
        max_length = self.max_sentence_length
        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)
        context_hidden = context.initHidden()
#         print('sentences', sentences)
        for i,sentence in enumerate(sentences):
            last = False
            if i + 1 == len(sentences):
                last = True
            input_variable = self.variableFromSentence(sentence=sentence)
            input_length = input_variable.size()[0]
            encoder_hidden = encoder.initHidden()

            encoder_outputs = Variable(torch.zeros(max_length, encoder.hidden_size))
            encoder_outputs = encoder_outputs.cuda() if use_cuda else encoder_outputs

            for ei in range(input_length):
                encoder_output, encoder_hidden = encoder(input_variable[ei],
                                                         encoder_hidden)
                encoder_outputs[ei] = encoder_outputs[ei] + encoder_output[0][0]

            decoder_input = Variable(torch.LongTensor([[self.SOS_token]]))  # SOS
            decoder_input = decoder_input.cuda() if use_cuda else decoder_input

            decoder_hidden = encoder_hidden

            # calculate context
            context_output,context_hidden = context(encoder_output,context_hidden)

            def decode_with_beam(decoder_inputs,decoder_hiddens,beam):
                new_decoder_inputs = []
                new_decoder_hiddens = []
                decoder_outputs = torch.FloatTensor().cuda() if use_cuda else torch.FloatTensor()
                #decoder_outputs_h = torch.FloatTensor()
                for i,decoder_input in enumerate(decoder_inputs):
                    decoder_output, decoder_hidden, decoder_attention = decoder(
                        decoder_input, decoder_hiddens[i], encoder_output, encoder_outputs,context_hidden)
                    #print decoder_output.data
                    #print decoder_outputs
                    decoder_outputs = torch.cat((decoder_outputs,decoder_output.data),1)
                    #decoder_outputs_h = torch.cat((decoder_outputs_h,decoder_output[0]),1)
                    new_decoder_hiddens.append(decoder_hidden)

                topv,topi = decoder_outputs.topk(beam)
                nis = list(topi[0])
                nh = [] # decoder_hidden
                for ni in nis:
                    nip = ni % len(self.word2id.keys()) # get the word id
                    #if nip == EOS_token:
                    #    continue # or break?
                    decoder_input = Variable(torch.LongTensor([[nip]]))
                    decoder_input = decoder_input.cuda() if use_cuda else decoder_input
                    new_decoder_inputs.append(decoder_input)
                    nh.append(new_decoder_hiddens[int((ni / len(self.word2id.keys())))])

                return new_decoder_inputs, nh,(nis[0] % len(self.word2id.keys()))

            decoder_inputs = [decoder_input]
            decoder_hiddens = [decoder_hidden]
            max_sent = 0
            for di in range(max_length):
                decoder_inputs,decoder_hiddens,ni = decode_with_beam(decoder_inputs,decoder_hiddens,beam)
                
                if last:
                    if ni == self.EOS_token:
                        decoded_words.append('<eos>')
                        max_sent += 1
                    else:
                        decoded_words.append(self.id2word[ni])
                        
                    if max_sent == 2:
                        break

        return decoded_words

    def evaluateRandomly(self, encoder, decoder, context, n=5):
        for i in range(n):
            group = random.choice(self.groups)
            for gr in group:
                print('>', gr)
            output_words = self.evaluate(encoder, decoder, context, group[:-1])
            output_sentence = ' '.join(output_words)
            print('<', output_sentence)
            print('*' * 50)
            
    def generateResponse(self, text='', n=5, encoder=None, decoder=None, context=None):
        encoder = self.encoder_model
        decoder = self.decoder_model
        context = self.context_model
        print(u'text>>', text)
        output_words = self.evaluate(encoder, decoder, context, text)
        output_sentence = u' '.join(output_words)
        print('<', output_sentence)
        print('*' * 50)
        return output_sentence

In [19]:
def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [ ]:

# prepare data
print ("loading data")
hredQA = HRED_QA(groups=groups,id2word=id2word, word2id=word2id,
                 hidden_size=EMBEDDING_DIM, beam=1,
        encoder_file='data/HRED_300_emb_encoder.model',
        decoder_file='data/HRED_300_emb_decoder.model',
        context_file='data/HRED_300_emb_context.model')

hredQA.trainIters(print_every=2000, plot_every=1000, n_epochs=50000, evaluate_every=10000)



In [23]:
print ("loading data")
hredQA = HRED_QA(groups=groups,id2word=id2word, word2id=word2id,
                 hidden_size=EMBEDDING_DIM, beam=1,
        encoder_file='data/HRED_300_emb_encoder.model',
        decoder_file='data/HRED_300_emb_decoder.model',
        context_file='data/HRED_300_emb_context.model')

loading data
loading encorder


In [24]:
# df = pd.read_csv('NLG_Hansard_generated_responses.tsv', sep='\t')

df = pd.read_csv('data/Hansard_test.csv')

HRED_pred = []
for idx, row in df.iterrows():
#     print(row['Q'].split('.')[:-1])
    try:
        HRED_pred.append( hredQA.generateResponse(text=row['Q'].split(' . ')))
    except Exception: 
        HRED_pred.append('nothing')
        print('FAIL', row['Q'])
        
df['HRED_PRED'] = HRED_pred

df.to_csv('NLG_Hansard_generated_responses.tsv', sep='\t')

df.tail()    

text>> ['sos while $212,000 is not much to the owners of family fortunes and french villas, it is several years of wages for middle-class canadians. eos sos how can these liberals justify sticking hard-working taxpayers with a quarter million dollar bill for nothing more than some fancy photo images on a budget cover? eos']
FAIL sos while $212,000 is not much to the owners of family fortunes and french villas, it is several years of wages for middle-class canadians. eos sos how can these liberals justify sticking hard-working taxpayers with a quarter million dollar bill for nothing more than some fancy photo images on a budget cover? eos
text>> ["sos that represents a down payment on a french villa or half of a mercedes roadster. eos sos how can the liberals justify sticking taxpayers with $400,000 for two years' worth of budget cover pages? eos"]


/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:84: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/home/paperspace/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:97: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


< sos the hon . <eos> sos member for his question . <eos>
**************************************************
text>> ['sos member for louis-hbert says that his government is working for the less fortunate. eos sos what does he have to say to them after spending $200,000 on a piece of a paper? eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> . <eos>
**************************************************
text>> ['sos now her lawyers are going after crupi for the money, but that was money stolen from the mouths of children. eos sos what steps will she take to make it right for the children of kashechewan and right for the people of canada? eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> . <eos>
**************************************************
text>> ['sos how can we truly achieve reconciliation when mtis and non-status are being told by the liberals that they do not exist and that their suffering is not worthy of recognition and compensation? eos sos 

**************************************************
text>> ['sos the hon. eos sos leader of the opposition. eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> <unk> <unk> . <eos>
**************************************************
text>> ['sos we want to ensure that intergenerational transfers of businesses continue. eos sos we will support-- eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> . <eos>
**************************************************
text>> ['sos my question is simple. eos sos why did he not do so? eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> . <eos>
**************************************************
text>> ['sos i know my definition of conflict of interest', 'eos sos i would like the minister to share his definition. eos']
< sos mr . speaker i would like to thank my colleague for his question . <eos> sos the hon . <eos>
***********************************************

< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> <unk> . <eos>
**************************************************
text>> ['sos under conservative governments, the americans respected our system of supply management, but the liberal government is literally ready to sell the farm. eos sos when will that member and the government stop saying one thing to canadian farmers and another thing to their friends in washington? eos']
< sos the hon . <eos> sos member for her question . <eos>
**************************************************
text>> ['sos therefore, my question for the minister is simple. eos sos we know you do not have a plan to fix phoenix, but when will you have a plan to have a plan? eos']
< sos the hon . <eos> sos member for his question . <eos>
**************************************************
text>> ['sos some hon. eos sos members: oh, oh! eos']
< sos the hon . <eos> sos member for his question . <eos>
**************************************************
text

...

< sos mr . speaker i sos the to the the sos <eos> sos member for her question . <eos>
**************************************************
text>> ['sos was the prime minister aware that his national security adviser, daniel jean, was going to give a media briefing during his recent trip to india? eos sos yes or no? eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> <unk> <unk> . <eos>
**************************************************
text>> ["sos mr. speaker, i will repeat my question, which is very straightforward and has nothing to do with the prime minister's last four answers. eos sos was the prime minister aware that his national security adviser, daniel jean, was going to give a media briefing during his recent trip to india, yes or no? eos"]
FAIL sos mr. speaker, i will repeat my question, which is very straightforward and has nothing to do with the prime minister's last four answers. eos sos was the prime minister aware that his national security adviser, da

text>> ['sos whether for the aluminum smelter in saguenay, the port of montreal, or the arctic, there is a shortage of efficient icebreakers like the aiviq, even though a solution is easily within reach. eos sos when will the prime minister put the davie shipyard workers back to work? eos']
< sos the hon . <eos> sos member for rosemontla <unk> <unk> . <eos>
**************************************************
text>> ['sos some hon. eos sos members: oh, oh! eos']
< sos the hon . <eos> sos member for his question . <eos>
**************************************************
text>> ['sos mr. speaker, i believe that if you seek it, you will find unanimous consent for the following motion: that the government of canada intervene with the german and spanish governments to call for the release of the following political prisoners: carles puigdemont, jordi turull, ral romeva, carme forcadell, dolors bassa, and josep rull. eos']
FAIL sos mr. speaker, i believe that if you seek it, you will find unan

< sos the hon . <eos> sos member for rosemontla <unk> <unk> <unk> <unk> <unk> . <eos>
**************************************************
text>> ["sos while canadians are waiting for infrastructure projects that are years overdue, while canadians wait for housing, liberals are stacking the board with wealthy supporters. eos sos why does the government not put canadians' interests ahead of the liberal party and replace the discredited infrastructure bank with public investment in infrastructure? eos"]
FAIL sos while canadians are waiting for infrastructure projects that are years overdue, while canadians wait for housing, liberals are stacking the board with wealthy supporters. eos sos why does the government not put canadians' interests ahead of the liberal party and replace the discredited infrastructure bank with public investment in infrastructure? eos
text>> ['sos after 10 years of stalled funding and neglect, in 2016 with our first budget, our government immediately began rebuildin

< sos the hon . <eos> sos member for his question . <eos>
**************************************************
text>> ['sos that is his choice. eos sos i have every right to ask questions that make the government uncomfortable and i am going to continue to do it whether those members like it or not. eos']
< sos the hon . <eos> sos member for his question . <eos>
**************************************************


,Q,A,HRED_PRED
1995,"sos yesterday, the prime minister and the public safety minister suggested that the content of that media briefing was classified. eos sos can the prime minister inform us who in his office helped orchestrate the release of classified information to the media? eos","sos mr. speaker, of course, at no point did members of the public service ever reveal classified information to the media, nor would they. eos sos the issue is that the leader of the opposition so wants to be able to play political games with this issue that he plugs his ears, refuses to know th...",sos order please . <eos> sos the hon . <eos>
1996,"sos the prime minister has just affirmed that classified information was not provided to the media. eos sos why, then, can that same information not be provided to the house? eos","sos mr. speaker, only stephen harper's conservative party would think that giving information to the media is somehow hiding information from canadians. eos sos the question is: why does the leader of the opposition not want to know the truth? eos",sos order please . <eos> sos the hon . <eos>
1997,"sos mr. speaker, in a healthy and prosperous society, it is essential to ensure that everyone can participate in the economy on a level playing field. eos sos can the minister of status of women tell us what we are doing in budget2018 to further empower women so that they have equal opportunitie...","sos mr. speaker, i thank my colleague from vimy for her leadership in advancing gender equality. eos sos by investing in women, we will improve the economy for everyone. eos",nothing
1998,sos it is unacceptable language. eos sos could the minister please clarify what he was trying to say? eos,"sos mr. speaker, i was at the committee meeting yesterday. eos sos it is unfortunate that the opposition members make such personal attacks, when they have an opportunity to discuss budget 2018 and provide constructive criticism. eos",sos the hon . <eos> sos member for his question . <eos>
1999,sos that is his choice. eos sos i have every right to ask questions that make the government uncomfortable and i am going to continue to do it whether those members like it or not. eos,"sos mr. speaker, we often see in the house a type of selective amnesia. eos sos i would like to remind the house that the cabinet of the previous prime minister, stephen harper, was far from gender-balanced. eos",sos the hon . <eos> sos member for his question . <eos>


In [ ]:
for i,sentence in enumerate('this is an sentence'.split(' ')):
    print(i, sentence)